# Optimal Transport with Linear Programming

In this assignment we will see how to solve the discrete optimal transport
problem (in the case of measures that are sums of Diracs) using linear
programming.

You need to install [CVXPY](https://www.cvxpy.org/). We recommend to run this in Google Colab, which has it pre-installed.

However, if running locally, you can simply do ```uv sync``` and it should install all the required dependencies (get to know [uv](https://docs.astral.sh/uv/)).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cp

Optimal Transport of Discrete Distributions
------------------------------------------

We consider two discrete distributions
$$
\alpha = \sum_{i=1}^n a_i \delta_{x_i} \quad \text{and} \quad
\beta = \sum_{j=1}^m b_j \delta_{y_j}.
$$

where $n, m$ are the number of points, $\delta_x$ is the Dirac at
location $x \in \mathbb{R}^d$, and $(x_i)_i$, $(y_j)_j$ are the positions of the Diracs in $\mathbb{R}^d$.

Consider the dimensions $n = 60$, $m=80$ for the coulds.

In [ ]:
n = 60
m = 80

**Exercise 1.** Generate the point clouds $X=(x_i)_{i=1}^n$ and $Y=(y_j)_{j=1}^m$ as follows:
- Each point $x_i$, $y_i$ will be in $\mathbb{R}^2$
- Each $x_i$ must be i.i.d. sampled from a Gaussian random variable $N([0,0], \sigma^2 Id_2)$ with $\sigma = 0.3$
- Each $y_i$ must be sampled from a **mixture of gaussians** as follows:
  - Half of the points must be i.i.d. gaussians centered at [0, 1.6] with std $0.5$.
  - One quarter of the points must be i.i.d. gaussians centered at [-1, -1] with std $0.3$.
  - One quarter of the points must be i.i.d. gaussians centered at [1, 1] with std $0.3$.

In [ ]:
# FILL IN WITH YOUR ANSWER
# X should be an array of shape (2, n) and Y should be an array of shape (2, m)

**Exercise 2.** Generate random weights $a,b$ for our discrete distributions. For this, simply normalize (to get a probability distribution) a sample of standard gaussians in $\mathbb{R}^n$ and $\mathbb{R}^m$, respectively.

In [ ]:
# FILL IN WITH YOUR ANSWER
# a should be an array of shape (n,) and b should be an array of shape (m,)

We give you the following helper function for displaying point clouds.

In [ ]:
myplot = lambda x,y,ms,col: plt.scatter(x,y, s=ms*20, edgecolors="k", c=col, linewidths=2)

Use the following code to display the point clouds. The size of each dot is proportional to its probability density weight.

In [ ]:
plt.figure(figsize = (10,7))
plt.axis("off")
for i in range(len(a)):
    myplot(X[0,i], X[1,i], a[i]*len(a)*10, 'b')
for j in range(len(b)):
    myplot(Y[0,j], Y[1,j], b[j]*len(b)*10, 'r')
plt.xlim(np.min(Y[0,:])-.1,np.max(Y[0,:])+.1)
plt.ylim(np.min(Y[1,:])-.1,np.max(Y[1,:])+.1)
plt.show()

**Exercise 3.** Describe the point cloud. How is $\alpha$ different from $\beta$ ?

[FILL IN]

**Exercise 4.** Compute the cost matrix $C_{i,j} := \|x_i-x_j\|^2$.

In [ ]:
def distmat(x, y):
    """Compute squared Euclidean distance matrix C where C[i,j] = ||x_i - y_j||^2
    Parameters:
        x: array of shape (2, n) containing source points
        y: array of shape (2, m) containing target points
    Returns:
        C: array of shape (n, m) containing squared distances
    """
    # FILL IN
    pass

In [ ]:
C = distmat(X,Y)

We will use `cvxpy` for solving this linear program. See some documentation [here](https://www.cvxpy.org/).

**Exercise 5.** Solve the linear program using`cvxpy`.

For this, know that `cvxpy` works by having you declare three things: a variable, a list of constraints, and an objective. You then wrap them into a cp.Problem and call .solve().

**a) Variable.** Declare a matrix-valued optimization variable $P \in \mathbb{R}^{n \times m}$ using `cp.Variable((n, m))`.

In [ ]:
P = # FILL IN

**b) Constraints.** We know that the set of discrete couplings between $\alpha$ and $\beta$ is
$$
U(a, b) := \left\{ P \in \mathbb{R}_+^{n \times m} \;\middle|\;
\forall i, \sum_j P_{i,j} = a_i, \quad
\forall j, \sum_i P_{i,j} = b_j \right\}.
$$
Encode this as a Python list `constraints` that allow to define the domain. Note that `cp.sum(P, axis=1, keepdims=True)` computes row sums, `cp.sum(P, axis=0, keepdims=True).T` computes column sums and `P>=0` defines positivity.

In [ ]:
constraints = [
# FILL IN
]

**c) Objective.** We know that the Kantorovich formulation of optimal transport reads
$$
    P^\star \in \underset{P \in U(a, b)}{\arg\min} \sum_{i,j} P_{i,j} C_{i,j}.
$$

Write down this objective function by using:
- `cp.multiply` for elementwise multiplication
- `cp.sum` to sum all entries.
- Define the objective function as `objective = cp.Minimize(...the sum of elementwise products...)`.

In [ ]:
objective = # FILL IN

**d) Solve**. Combine the objective and constraints into an object `prob = cp.Problem(objective, constraints)` and call .solve() to recover the optimal value.

In [ ]:
result = # FILL IN

An optimal coupling $P^\star$ can be shown to be a sparse matrix
with less than $n+m-1$ non zero entries. An entry $P_{i,j}^\star \neq 0$
should be understood as a link between $x_{i}$
and $y_{j}$ where an amount of mass equal to $P_{i,j}^\star$ is transfered.

**Exercise 6.** Check that the number of non-zero entries in $P^\star$ is $n+m-1$. Beware that we are using an interior point method here, so that entries of $P^\star$ are nevery exactly 0.

In [ ]:
# FILL IN

We now visualize the solution coupling.

In [ ]:
plt.figure(figsize = (5,5))
plt.imshow(P.value);

We also display the connexion defined by the optimal coupling.

In [ ]:
I,J = np.nonzero(P.value>1e-5)
plt.figure(figsize = (10,7))
plt.axis('off')
for k in range(len(I)):
    h = plt.plot(np.hstack((X[0,I[k]],Y[0,J[k]])),np.hstack(([X[1,I[k]], Y[1,J[k]]])),'k', lw = 2)
for i in range(len(a)):
    myplot(X[0,i], X[1,i], a[i]*len(a)*10, 'b')
for j in range(len(b)):
    myplot(Y[0,j], Y[1,j], b[j]*len(b)*10, 'r')
plt.xlim(np.min(Y[0,:])-.1,np.max(Y[0,:])+.1)
plt.ylim(np.min(Y[1,:])-.1,np.max(Y[1,:])+.1)
plt.show()

**Exercise 7.** What do you see? Is this coupling defined by a transport map?

[FILL IN]

Displacement Interpolation
--------------------------
**[NO exercises on this section]**

For any $t \in [0,1]$, one can define a distribution $\mu_t$ such that the map $t \mapsto \mu_t$ describes a geodesic in the Wasserstein space.

Since the $W_2$ distance is a geodesic distance, this geodesic path solves the following variational problem:
$$
\mu_t = \underset{\mu}{\arg\min} \, (1 - t) W_2(\alpha, \mu)^2 + t W_2(\beta, \mu)^2.
$$

This can be understood as a generalization of the classical Euclidean barycenter to the barycenter of probability distributions. Indeed, in the special case where $\alpha = \delta_x$ and $\beta = \delta_y$, one obtains $\mu_t = \delta_{x_t}$ with
$$
x_t = (1 - t)x + t y.
$$

Once the optimal coupling $P^\star$ has been computed, the interpolated distribution is given by
$$
\mu_t = \sum_{i,j} P^\star_{i,j} \, \delta_{(1 - t)x_i + t y_j}.
$$

We can visualize this by finding the $i,j$ with non-zero $P_{i,j}^\star$.

In [ ]:
I,J = np.nonzero(P.value>1e-5)
Pij = P.value[I,J]

And we now display the evolution of $\mu_t$ for a varying value of $t \in [0,1]$.

In [ ]:
plt.figure(figsize =(15,10))
tlist = np.linspace(0, 1, 6)
for i in range(len(tlist)):
    t = tlist[i]
    Xt = (1-t)*X[:,I] + t*Y[:,J]
    plt.subplot(2,3,i+1)
    plt.axis("off")
    for j in range(len(Pij)):
        myplot(Xt[0,j],Xt[1,j],Pij[j]*len(Pij)*6,[[t,0,1-t]])
    plt.title("t = %.1f" %t)
    plt.xlim(np.min(Y[0,:])-.1,np.max(Y[0,:])+.1)
    plt.ylim(np.min(Y[1,:])-.1,np.max(Y[1,:])+.1)
plt.show()

Optimal Assignement
-------------------
In the case where $n = m$ and the weights are uniform, i.e., $a_i = 1/n$ and $b_j = 1/n$, one can show that there exists at least one optimal transport coupling that is a permutation matrix. This property arises from the fact that the extremal points of the transport polytope $U(1, 1)$ are permutation matrices.

This means there exists an optimal permutation $\sigma^\star \in \Sigma_n$ such that
$$
P^\star_{i,j} =
\begin{cases}
1 & \text{if } j = \sigma^\star(i), \\
0 & \text{otherwise}.
\end{cases}
$$

Here, $\Sigma_n$ denotes the set of all permutations (bijections) of $\{1, \ldots, n\}$.

This permutation solves the so-called *optimal assignment problem*:
$$
\sigma^\star \in \underset{\sigma \in \Sigma_n}{\arg\min}
\sum_{i=1}^n C_{i, \sigma(i)}.
$$

**Exercise 8.** Repeat all the previous steps to solve this problem. Namely:
- Set $n=40=m$.
- Compute the same point clouds as in **Ex.1**.
- Set **uniform** weights on the particles (**unlike Ex.2**, no randomness here).
- Compute the cost matrix (you can use the functions you built!).
- Solve the Optimal Transport problem using `cvxpy`.
- Visualize the optimal assignment, as in **Ex.7**
- Visualize the displacement interpolation, as in the previous section.

Comment on the obtained results and on the key differences with the case $n\neq m$.

In [ ]:
# FILL IN

Analysis: [FILL IN]